# Introduction

put stuff here

# PART 1 : Data Collection
In this section we will collect all the data we will need to begin studying and analyzing depression across U.S cities and it's possible causes

Let's start off by importing all the Python libraries we will need.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import requests
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}

List All Data sources here and talk about what information they give us

Before we start getting the necessary dataframes, here we will create a list of states. We need this list of states to fetch data from the cdc chronicdata API. The dataframe we will be using for depression information about cities is very big and for this reason we will fetch parts of it at a time, using the state abbreviations (from the list below) to get certain parts.

In [2]:
# List of state abbreviations
states = ['AL', 'AK', 'AZ' ,'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
 'NH', 'NJ', 'NM', 'NY', 'DC', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


Below we fetch the Depression and Mental Health Dataframe from the CDC chronicdata api. To do this we will be using HTTP requests using python's requests library. To do this in a more efficient way, what we do here is go through each state abbreviation and get the json representation of the data for that state. In the request we specify the measure id, both MHTLH(mental health) and DEPRESSION and then specify a limit of 4000 rows. We do 4000 rows because no state has any more than 4000. After we get the json response we append it to a list.



In [3]:
list = []
x = 0
#df = pd.json_normalize(res.json())
for state in states:
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=MHLTH&$limit=4000&stateabbr={state}')
    list.append(res.json())
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=DEPRESSION&$limit=4000&stateabbr={state}')
    list.append(res.json())
    
    

Now we will create the pandas dataframe by going through the list of json files and appending them

In [4]:
df = pd.json_normalize(list[0])
df.set_index('locationid')
for i in range(1,len(list)):
    curr = pd.json_normalize(list[i])
    df = df.append(curr)


Once we have gotten the data frame we need to clean it up a bit. There are lots of columns that we don't really need so we can just delete them. We will also rename a few of the columns. 

In [5]:
# Cleaning up DF a little bit

df = df.rename(columns={'year': 'Year', 'statedesc': 'State', 'locationname': 'City', 'measure': 'Symptom', 'data_value': 'Percent', 'stateabbr': 'State Abbr'})
del df['datasource']
del df['measureid']
del df['geolocation.coordinates']
del df['data_value_footnote_symbol']
del df['data_value_footnote']
del df['low_confidence_limit']
del df['high_confidence_limit']
del df['categoryid']
del df['datavaluetypeid']

df['Affected Population'] = df['totalpopulation'].astype('int') * df['Percent'].astype('float')

Next up we will import the income data set. Using the link from above you can download the csv file for this dataset. Once downloaded we will read it into a Pandas dataframe. As with the other dataframe, we will delete unneccesary columns and rename a few of the columns. In this dataframe there are rows of cities that actually repeat. To fix this issue we will use groupby and group together rows with the same city name. After doing so we aggregate the grouped columns using the average and this will become the dataframe.

In [6]:
# Import Income Data Set
income_df = pd.read_csv("kaggle_income.csv", encoding='latin-1')
income_df['loc_fullname'] = income_df['City'] + ', ' + income_df['State_Name']

# delete unnecessary columns
del income_df['id']
del income_df['Type']
del income_df['County']
del income_df['Place']
del income_df['Primary']
del income_df['Lat']
del income_df['Lon']
del income_df['sum_w']
del income_df['Stdev']
del income_df['Area_Code']
del income_df['ALand']
del income_df['AWater']


# rename columns
income_df = income_df.rename(columns = {'Mean':'Mean Income','Median':'Median Income', 'Zip_Code': 'Zip Code'})

# combine duplicate cities
grouped = income_df.groupby(['loc_fullname'], as_index=False)
income_df = grouped.agg('mean')
income_df.head(5)

,loc_fullname,State_Code,Zip Code,Mean Income,Median Income
0,"Abbeville, Alabama",1.0,36310.0,40518.000000,25216.000000
1,"Abbeville, Louisiana",22.0,70510.0,30647.000000,25628.333333
2,"Abbeville, South Carolina",45.0,29620.0,45585.666667,32245.000000
3,"Abbotsford, Wisconsin",55.0,54405.0,58254.000000,44919.000000
4,"Aberdeen, Maryland",24.0,21001.0,96480.000000,80370.000000


Next up is the population dataset. Just as with the income dataset, we download this dataset as a csv file and read this file into a Pandas dataframe directly. We again delete uneccesary columns and rename columns. For this dataframe we will also create a few new columns for location fullname, and county name, by combing different columns together. 

In [7]:
# import us cities dataset
cities_df = pd.read_csv('uscities.csv')
del cities_df['city_ascii']
del cities_df['lat']
del cities_df['lng']
del cities_df['source']
del cities_df['military']
del cities_df['incorporated']
del cities_df['timezone']
del cities_df['ranking']
del cities_df['id']

cities_df = cities_df.rename(columns={'population': 'Total Population', 'city': 'City', 'state_id': 'State Abbr',
                                     'state_name': 'State', 'county_name': 'County', 'density': 'Density'})
cities_df['loc_fullname'] = cities_df['City'] + ', ' + cities_df['State']
cities_df['County'] = cities_df['County'] + ' County'

cities_df.head(5)

,City,State Abbr,State,county_fips,County,Total Population,Density,zips,loc_fullname
0,New York,NY,New York,36061,New York County,18713220,10715,11229 11226 11225 11224 11222 11221 11220 1138...,"New York, New York"
1,Los Angeles,CA,California,6037,Los Angeles County,12750807,3276,90291 90293 90292 91316 91311 90037 90031 9000...,"Los Angeles, California"
2,Chicago,IL,Illinois,17031,Cook County,8604203,4574,60018 60649 60641 60640 60643 60642 60645 6064...,"Chicago, Illinois"
3,Miami,FL,Florida,12086,Miami-Dade County,6445545,5019,33129 33125 33126 33127 33128 33149 33144 3314...,"Miami, Florida"
4,Dallas,TX,Texas,48113,Dallas County,5743938,1526,75287 75098 75233 75254 75251 75252 75253 7503...,"Dallas, Texas"


The final dataset we will be using is the temperature dataset. We read the csv file into a new Pandas dataframe. After doing this we will create a new column called state, which is just the location id column but only the first two characters.

In [8]:
# lets also do temperature by county, import from this dataset

temperature = pd.read_csv('temp_by_county.csv')

temperature = temperature.rename(columns={'Location': 'County', '1901-2000 Mean': 'Annual Avg Temp F'})

# this data is formatted pretty weird
# lets add a state column so we can more reasily merge it back onto our main df
temperature['State Abbr'] = temperature['Location ID'].apply(lambda x: x[:2])
temperature.head(5)

,Location ID,County,Value,Rank,Anomaly (1901-2000 base period),Annual Avg Temp F,State Abbr
0,AL-001,Autauga County,53.0,48,-1.3,54.3,AL
1,AL-003,Baldwin County,57.6,61,-0.7,58.3,AL
2,AL-005,Barbour County,54.6,47,-1.2,55.8,AL
3,AL-007,Bibb County,51.2,44,-1.6,52.8,AL
4,AL-009,Blount County,50.3,61,-0.3,50.6,AL


# Part 2 : Data Proccessing

In this part we will combine and modify the dataframes so we can analyze and study the data more easily.

Now going back to the depression and mental health dataframe we will modify it a bit to make it easier to merge all the dataframes together. We first create a new column called loc_fullname, which is just a combination of the city and state columns. We then merge the three other dataframes using various columns to do so. That should result in the following.  

In [9]:
# Create a column that holds: {City}, {State}
# We will merge using this as the key. Then we will delete this (because we already have columns for state and cities)

df['loc_fullname'] = df['City'] + ', ' + df['State']
df = df.merge(income_df[['loc_fullname', 'Mean Income', 'Median Income', 'Zip Code']], on='loc_fullname')
df = df.merge(cities_df[['loc_fullname', 'Total Population', 'County', 'Density']], on='loc_fullname')
df = df.merge(temperature, on=['State Abbr', 'County'])
del df['loc_fullname']

Now that we have a dataframe holding all the information we need, it is time to clean up a bit. The formatting of the dataframe as it is now is very clunky with columns specifying what values represent with other columns as the actual values. We will create columns with the actual values, with the type of data those values represent being the column names. At the same time we will change the type of these columns to double so that they are easier to work with. 

In [10]:
df['Mental Health Age-Adjusted Percent'] = (df.loc[((df['short_question_text'] == 'Mental Health') & (df['data_value_type'] == 'Age-adjusted prevalence'))]['Percent']).astype('double')
df['Mental Health Crude Percent'] = (df.loc[((df['short_question_text'] == 'Mental Health') & (df['data_value_type'] == 'Crude prevalence'))]['Percent']).astype('double')
df['Depression Age-Adjusted Percent'] = (df.loc[((df['short_question_text'] == 'Depression') & (df['data_value_type'] == 'Age-adjusted prevalence'))]['Percent']).astype('double')
df['Depression Crude Percent'] = (df.loc[((df['short_question_text'] == 'Depression') & (df['data_value_type'] == 'Crude prevalence'))]['Percent']).astype('double')


In [17]:
df.head(5)
df.columns.values

array(['Year', 'State Abbr', 'State', 'City', 'County', 'Mean Income',
       'Median Income', 'Annual Avg Temp F', 'Total Population',
       'Density', 'Affected Population',
       'Mental Health Age-Adjusted Percent',
       'Mental Health Crude Percent', 'Depression Age-Adjusted Percent',
       'Depression Crude Percent'], dtype=object)

Lets delete some unnecessary columns and rename some columns as well. After this we will combine the repeated city columns into one where all the mental health and depression data will be together.

In [12]:
del df['Symptom']
del df['data_value_unit']
del df['data_value_type']
del df['Percent']
del df['short_question_text']
del df['Location ID']
del df['Value']
del df['Rank']
del df['Anomaly (1901-2000 base period)']
del df['category']
del df['totalpopulation']
df['Zip Code'] = df['Zip Code'].astype('int')

In [13]:
# Aggregate rows to have one instance of each and information on mental health and depression rates
grouped = df.groupby(by=['Year','State', 'State Abbr','City','locationid','Mean Income','Median Income','Zip Code'
                         ,'Total Population', 'County', 'Annual Avg Temp F'], as_index=False, dropna=False)
df = grouped.agg('sum')


# Reordering columns
df = df[['Year', 'State Abbr', 'State', 'City', 'County', 'Mean Income', 'Median Income', 'Annual Avg Temp F'
         ,'Total Population', 'Density', 'Affected Population', 'Mental Health Age-Adjusted Percent', 'Mental Health Crude Percent', 'Depression Age-Adjusted Percent',
        'Depression Crude Percent']]

In [14]:
df.head(5)

,Year,State Abbr,State,City,County,Mean Income,Median Income,Annual Avg Temp F,Total Population,Density,Affected Population,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
0,2019,AL,Alabama,Abbeville,Henry County,40518.000000,25216.000000,56.8,2560,252,233587.2,20.2,18.3,24.8,23.6
1,2019,AL,Alabama,Adamsville,Jefferson County,53383.000000,47460.000000,51.7,4281,260,378943.6,18.8,18.1,23.6,23.3
2,2019,AL,Alabama,Alabaster,Shelby County,81118.333333,149711.333333,52.1,33487,2048,2446626.0,15.4,15.6,23.3,23.7
3,2019,AL,Alabama,Albertville,Marshall County,47199.500000,160358.500000,50.5,37775,1252,2030302.8,19.4,19.5,28.2,28.3
4,2019,AL,Alabama,Alexander City,Tallapoosa County,40286.000000,31306.000000,52.8,14317,524,1342988.4,19.5,18.6,26.1,25.5


Now we will set the index of the dataframe as locationid. Something you may notice is that there are rows with mean income of 0. What this means is that there were no records of income for these cities. We have two options with this issue, either delete these rows, or to make all the 0s into NaN values. We decided to delete these rows because they don't contribute too much to our data. The dataframe we are using has over 8000 rows so dropping some of them shouldn't affect our analysis.

In [15]:
df.set_index('locationid',inplace = True)

KeyError: "None of ['locationid'] are in the columns"

In [ ]:
df.loc[df['Mean Income'].astype('int') < 1000]
# As per our data source, 0 income means that there are no records of income for these cities. 
# We can either make these values NAN or completely get rid of these rows. 
# We decided to just delete these rows as none of them are extremely 'important' cities. 

In [ ]:
df = df.drop(df[df['Mean Income'].astype('int') < 1000].index)
df.head(5)

In [ ]:
# That was all collection and processing. Now exploratory analysis and Visualization

# Some things we could possibly explore in analysis
    # most depressed cities
    # avg depression/mental health rate by city
    # state with worst depression, state with worst mental health
    # income vs depression rates
    # population density vs depression/mental health rates
    # temp vs depression/mental health rates
    # for every city, mental health rates vs depression rates, similar?
    # maybe try using folium, geopandas, basemap. Couldn't get any to work atm
        # talk about regions that seem to have higher rates of mental illness/depression
        # if we dont do this we could also keep longitude/latitude data, plot a distribution